In [1]:
from utils import petab as pet
import numpy as np
import utils.sbml as sbml
import utils.amici as am
from models.sbml_models import CRP2_CPE
%load_ext autoreload
%autoreload 2

The tests rely on tellurium to construct the models
Since tellurium is not installed the tests can't be run
If you want to run the tests, pip install tellurium first


### Define SBML/AMICI Model

In [3]:
observables_df = pet.define_observables({
    'xA': 'xA',
    'xB': 'xB',  
}, noise_value=0.02)

# Load SBML/AMICI model
sbml_model_filepath = sbml.write_model(CRP2_CPE)

# validator = sbml.validateSBML(ucheck=False)
# validator.validate(sbml_model_filepath)
# model_name, model_output_dir = am.compile_amici_model(
#     sbml_model_filepath, observables_df, verbose=False
# )
model_name = 'CRP2_CPE'
model_output_dir = '/SBML/amici_models/CRP2_CPE/'
model = am.load_amici_model(model_name, model_output_dir)


Creating SBML model (CRP2_CPE).
Creating CRP parameters.


### Define Measurements

In [10]:
# Define model parameters
true_params = {
    # 'kpAA': 1.0,
    "rA": 5.0,
    "rB": 1.0,
    "rX": 1.0,
    "KAA": 0.0,
    "KAB": 0.0,
    "KBA": 0.0,
    "KBB": 0.0,
}
model = am.set_model_parameters(model, true_params)

# Define conditions
fA = np.array([0.1, 0.25, 0.5, 0.75, 0.9], dtype=float)
cM = np.array([1.0, 1.0, 1.0, 1.0, 1.0], dtype=float)
A0 = fA * cM
B0 = (1 - fA) * cM

# Define conditions dataframe
conditions_df = pet.define_conditions(
    init_conditions={
        "A0": list(A0),
        "B0": list(B0),
    }
)

# Define conversion values
timepoints = list(np.arange(0, 1, 0.1, dtype=float))
# print(timepoints)

measurements_df, rdatas = pet.define_measurements(
    amici_model=model,
    timepoints=timepoints,
    conditions_df=conditions_df,
    observables_df=observables_df,
    obs_sigma=0.0,
    meas_sigma=0.005,
    debug_return_rdatas=True,
)

measurements_df

2025-01-09 21:25:52.821 - amici.swig_wrappers - DEBUG - [idas_ic:IDAICFailFlag:CONV_FAILURE] Newton/Linesearch algorithm failed to converge.
2025-01-09 21:25:52.822 - amici.swig_wrappers - ERROR - [OTHER] AMICI simulation failed: IDA routine IDACalcIC failed with error code -4. 


,observableId,simulationConditionId,time,measurement
0,obs_xA,c_0,0.0,0.000000
1,obs_xA,c_0,0.1,0.131771
2,obs_xA,c_0,0.2,0.257730
3,obs_xA,c_0,0.3,0.377659
4,obs_xA,c_0,0.4,0.491286
...,...,...,...,...
95,obs_xB,c_4,0.5,0.000000
96,obs_xB,c_4,0.6,0.000000
97,obs_xB,c_4,0.7,0.000000
98,obs_xB,c_4,0.8,0.000000


In [5]:
from models.cpe_models import CopolymerEquations as CPE

cpe = CPE.from_ratios(true_params.values(), fA0=0.5, M0=1.0)
cpe.t_eval = timepoints
cpe.solve_izu()

/SBML/src/models/cpe_models.py:102: RuntimeWarning: invalid value encountered in scalar divide
  FA = dAdt / (dAdt + dBdt)

KeyboardInterrupt

